In [5]:
import pandas as pd
from datetime import datetime
from math import radians, cos, pi, exp

In [6]:
solar_altitudes_df = pd.read_csv('./solar_altitudes.csv')
solar_altitudes_df

,location,經度,緯度,地點名稱,DateTime,方位角,太陽仰角
0,1,121.544444,23.899444,頂樓一號,2024-01-01 00:00:00,181.000,-88.366208
1,1,121.544444,23.899444,頂樓一號,2024-01-01 00:10:00,181.000,-86.202505
2,1,121.544444,23.899444,頂樓一號,2024-01-01 00:20:00,181.000,-83.943446
3,1,121.544444,23.899444,頂樓一號,2024-01-01 00:30:00,181.000,-81.666617
4,1,121.544444,23.899444,頂樓一號,2024-01-01 00:40:00,181.000,-79.383910
...,...,...,...,...,...,...,...
746635,17,121.600000,23.983333,自家倉庫,2024-10-31 23:10:00,185.125,-75.947259
746636,17,121.600000,23.983333,自家倉庫,2024-10-31 23:20:00,185.125,-77.567816
746637,17,121.600000,23.983333,自家倉庫,2024-10-31 23:30:00,185.125,-78.921182
746638,17,121.600000,23.983333,自家倉庫,2024-10-31 23:40:00,185.125,-79.899069


In [7]:
# 確保 '時間' 欄位為 datetime 格式
solar_altitudes_df['DateTime'] = pd.to_datetime(solar_altitudes_df['DateTime'])

# 計算日射量並加入到 DataFrame
def calculate_solar_irradiance(altitude_angle, day_of_year, atmospheric_coefficient=0.2):
    # 常數
    solar_constant = 1361  # 太陽常數 (W/m²)
    theta = radians(90 - altitude_angle)  # 天頂角 (rad)
    
    # 計算太陽直射輻射
    I0 = solar_constant * (1 + 0.033 * cos(2 * pi * day_of_year / 365))
    
    # 大氣質量
    if altitude_angle > 0:
        m = 1 / (cos(theta) + 0.15 * (93.885 - altitude_angle)**-1.253)
    else:
        m = float('inf')  # 當仰角為0或負數時，無輻射量
    
    # 水平面日射量
    if m < float('inf'):
        I = I0 * cos(theta) * exp(-atmospheric_coefficient * m)
    else:
        I = 0  # 太陽在地平線以下時
    
    return I

# 計算日射量
solar_altitudes_df['日射量 (W/m²)'] = solar_altitudes_df.apply(
    lambda row: calculate_solar_irradiance(row['太陽仰角'], row['DateTime'].timetuple().tm_yday), axis=1
)


In [8]:
solar_altitudes_df.to_csv("sundata.csv",index=False)